# analysis V4 - new basis code - Dec 6, 2023
- cleaned up the data QC and organizations a bit
- having two anndata and their spin offs - both FISH and proj data 
- flip an axis (y-axis) when plotting - not in data
- focus on L2/3 cells

TODO: 
- separate and organize plotting functions 
- organize the plots and generate more insights

In [ ]:
import numpy as np
import pandas as pd
import os 
import matplotlib.pyplot as plt
import umap
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib

import anndata 
import scanpy as sc
from scipy.stats import spearmanr

from scroutines import config_plots
from scroutines import basicu
from scroutines import miscu

In [ ]:
import sys
sys.setrecursionlimit(10000)
from scipy.cluster import hierarchy as sch

In [ ]:
def norm_data(adata):
    """This procedure is independnet of each gene
    norm by size (cell volume)
    norm by log2(1+)
    norm by zscore
    """
    # size
    med_size = adata.obs['area'].median()
    max_size = adata.obs['area'].max()
    min_size = adata.obs['area'].min()

    print(f"Min cell size {min_size:.1f} um^3\t  {np.power(min_size,1/3):.1f} um")
    print(f"Med cell size {med_size:.1f} um^3\t  {np.power(med_size,1/3):.1f} um")
    print(f"Max cell size {max_size:.1f} um^3\t  {np.power(max_size,1/3):.1f} um")

    size_factor = (adata.obs['area']/med_size).values


    # norm by size; by log2+1; by zscore
    mat_raw = np.array(adata.X)
    mat_nrm = mat_raw/size_factor.reshape(-1,1) # .divide(size_factor, axis=0)
    mat_log = np.log2(1+mat_nrm)
    mat_zsc = (mat_log - np.mean(mat_log, axis=0))/np.std(mat_log, axis=0)

    adata.layers['nrm'] = mat_nrm
    adata.layers['log'] = mat_log
    adata.layers['zsc'] = mat_zsc
    
    return # mat_raw, mat_nrm, mat_log, mat_zsc

In [ ]:
pth_dat = '/u/home/f/f7xiesnm/project-zipursky/easifish/lt172/proc/r12345v3/'
!ls $pth_dat

# input

In [ ]:
f1 = '/u/home/f/f7xiesnm/project-zipursky/easifish/results/lt172_analysis_results_241002.csv'
f2 = '/u/home/f/f7xiesnm/project-zipursky/easifish/results/lt172_analysis_results_241002.h5ad'

df = pd.read_csv(f1)
df

In [ ]:
adata = anndata.read(f2)
adata

# Viz

In [ ]:
df

In [ ]:
df.columns
adata

In [ ]:
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler

# Sample data
X = df[['z', 'y', 'x']].values

Y = df[['ptime']].values

# Standardize the data
scaler_x = StandardScaler()
scaler_y = StandardScaler()

X_std = scaler_x.fit_transform(X)
Y_std = scaler_y.fit_transform(Y)

# Define the CCA model
cca = CCA(n_components=1)

# Fit the model
cca.fit(X_std, Y_std)

# Transform the datasets
X_c, Y_c = cca.transform(X_std, Y_std)

# Print the canonical variates
print("Canonical Variates for X:\n", X_c)
print("Canonical Variates for Y:\n", Y_c)
print(cca.coef_)

In [ ]:
cca.x_weights_

In [ ]:
cca.y_weights_

In [ ]:
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler

# Sample data
X = df[['z', 'y', 'x']].values

Y = df[['ux', 'uy']].values

# Standardize the data
scaler_x = StandardScaler()
scaler_y = StandardScaler()

X_std = scaler_x.fit_transform(X)
Y_std = scaler_y.fit_transform(Y)

# Define the CCA model
cca = CCA(n_components=2)

# Fit the model
cca.fit(X_std, Y_std)

# Transform the datasets
X_c, Y_c = cca.transform(X_std, Y_std)

# Print the canonical variates
print("Canonical Variates for X:\n", X_c)
print("Canonical Variates for Y:\n", Y_c)
print(cca.coef_)

In [ ]:
print(cca.coef_)

In [ ]:
cca.x_weights_

In [ ]:
cca.y_weights_